In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
import pickle
from time_series_split import *

In [20]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

In [3]:
def calculate_aqi(pm25):
    ranges = [
        (0.0, 12.0, 0, 50),
        (12.1, 35.4, 51, 100),
        (35.5, 55.4, 101, 150),
        (55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300),
        (250.5, 500.0, 301, 500),
    ]
    for c_low, c_high, aqi_low, aqi_high in ranges:
        if c_low <= pm25 <= c_high:
            return round((aqi_high - aqi_low) / (c_high - c_low) * (pm25 - c_low) + aqi_low)
    return 500  # default nếu vượt ngưỡng

def aqi_class(aqi):
    if aqi <= 50: return 0
    elif aqi <= 100: return 1
    elif aqi <= 150: return 2
    elif aqi <= 200: return 3
    elif aqi <= 300: return 4
    else: return 5

In [4]:
combined_data = pd.read_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/add_AQI.csv')
combined_data.head()

,time,ID,pm25,lat,lon,SQRT_SEA_DEM_LAT,WSPD,WDIR,TMP,TX,...,NDVI,CO,HCHO,NO2,SO2,CLOUD,O3,AAI,AQI,AQI_Class
0,2020-01-01,19,116.949130,21.049750,105.741870,5.922647,1.136119,145.942749,20.811243,23.219995,...,0.000551,0.045586,0.000060,0.000072,0.000035,0.711612,0.101653,-0.836203,183,3
1,2020-01-01,79,105.103043,21.015250,105.800130,4.307231,1.136119,145.942749,20.811243,23.219995,...,-0.003006,0.041913,0.000107,0.000086,0.000056,0.761720,0.103635,-0.718537,177,3
2,2020-01-01,163,118.285100,21.024347,106.017288,4.988467,0.651509,145.395233,20.677492,23.029993,...,0.004388,0.042329,0.000108,0.000071,0.000026,0.867689,0.102144,-0.942304,183,3
3,2020-01-01,300,116.739130,21.023532,105.853941,4.865087,1.136119,145.942749,20.811243,23.219995,...,-0.001733,0.041913,0.000097,0.000085,0.000079,0.779294,0.102875,-0.692613,183,3
4,2020-01-02,19,76.856667,21.049750,105.741870,5.922647,2.744283,147.084442,22.176249,25.119989,...,0.016035,0.042298,0.000115,0.000082,-0.000345,0.705396,0.103488,-1.119681,162,3


In [5]:
# Tính cột AQI từ PM2.5
combined_data['AQI'] = combined_data['pm25'].apply(calculate_aqi)
# Gán nhãn lớp AQI
combined_data['AQI_Class'] = combined_data['AQI'].apply(aqi_class)

In [6]:
combined_data.head()

,time,ID,pm25,lat,lon,SQRT_SEA_DEM_LAT,WSPD,WDIR,TMP,TX,...,NDVI,CO,HCHO,NO2,SO2,CLOUD,O3,AAI,AQI,AQI_Class
0,2020-01-01,19,116.949130,21.049750,105.741870,5.922647,1.136119,145.942749,20.811243,23.219995,...,0.000551,0.045586,0.000060,0.000072,0.000035,0.711612,0.101653,-0.836203,183,3
1,2020-01-01,79,105.103043,21.015250,105.800130,4.307231,1.136119,145.942749,20.811243,23.219995,...,-0.003006,0.041913,0.000107,0.000086,0.000056,0.761720,0.103635,-0.718537,177,3
2,2020-01-01,163,118.285100,21.024347,106.017288,4.988467,0.651509,145.395233,20.677492,23.029993,...,0.004388,0.042329,0.000108,0.000071,0.000026,0.867689,0.102144,-0.942304,183,3
3,2020-01-01,300,116.739130,21.023532,105.853941,4.865087,1.136119,145.942749,20.811243,23.219995,...,-0.001733,0.041913,0.000097,0.000085,0.000079,0.779294,0.102875,-0.692613,183,3
4,2020-01-02,19,76.856667,21.049750,105.741870,5.922647,2.744283,147.084442,22.176249,25.119989,...,0.016035,0.042298,0.000115,0.000082,-0.000345,0.705396,0.103488,-1.119681,162,3


In [7]:
# folds = split_original_data()
folds = split_consolidated_data()
print(f"Number of folds: {len(folds)}")

Number of folds: 3


**Best paramters**

Sau khi tuning thì được Best parameters: {'n_estimators': 1200, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 15}


Trước khi tuning:  F1-score = 0.67
Sau khi tuning: F1-score = 0.998

In [8]:
params = {
    'n_estimators': 1200,
    'max_features': 'log2',
    'max_depth': 15,
    'min_samples_split': 15,
    'min_samples_leaf': 2
}

# params = {
#     'n_estimators': 100,
#     'max_features': 'log2',
#     'max_depth': 5,
#     'min_samples_split': 10,
#     'min_samples_leaf': 10
# }

In [9]:
# Biến lưu nhãn thật và dự đoán trên toàn bộ test sets
all_y_true = []
all_y_pred = []

In [10]:
# Danh sách để lưu kết quả từ mỗi fold
val_accuracies = []
test_accuracies = []
test_classification_reports = []

best_val_accuracy = 0
best_model = None

In [11]:
all_target_names = ['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy', 'Very Unhealthy', 'Hazardous']

In [ ]:
models = {
    "RandomForest": RandomForestClassifier(**params, random_state=43, class_weight='balanced'),
    "XGBoost": XGBClassifier(n_estimators=500, max_depth=10, learning_rate=0.1, objective='multi:softmax', num_class=6, eval_metric='mlogloss', use_label_encoder=False, random_state=43),
    "CatBoost": CatBoostClassifier(iterations=500, depth=10, learning_rate=0.1, loss_function='MultiClass', verbose=False, random_seed=43)
}

# Thêm mô hình ensemble
models["Ensemble"] = VotingClassifier(
    estimators=[
        ('rf', models["RandomForest"]),
        ('xgb', models["XGBoost"]),
        ('cat', models["CatBoost"])
    ],
    voting='hard'
)

In [23]:
for model_name, model in models.items():
    print(f"\n{'='*30}\n▶ Training Model: {model_name}\n{'='*30}")

    all_y_true = []
    all_y_pred = []
    val_accuracies = []
    test_accuracies = []
    test_classification_reports = []

    best_val_accuracy = 0
    best_model = None

    for i, fold in enumerate(folds):
        print(f"\nProcessing Fold {i+1}/{len(folds)}")

        train_data = fold['train']
        val_data = fold['validation']
        test_data = fold['test']

        feature_columns = train_data.columns[2:-2]
        X_train = train_data[feature_columns]
        y_train = train_data['AQI_Class']
        X_val = val_data[feature_columns]
        y_val = val_data['AQI_Class']
        X_test = test_data[feature_columns]
        y_test = test_data['AQI_Class']

        model.fit(X_train, y_train)

        train_acc = model.score(X_train, y_train)
        val_acc = model.score(X_val, y_val)
        test_acc = model.score(X_test, y_test)

        print(f"Train Accuracy: {train_acc:.4f} | Validation Accuracy: {val_acc:.4f} | Test Accuracy: {test_acc:.4f}")

        val_accuracies.append(val_acc)
        test_accuracies.append(test_acc)

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            best_model = model

        y_pred = model.predict(X_test)
        all_y_true.extend(y_test.tolist())
        all_y_pred.extend(y_pred.tolist())

        report = metrics.classification_report(y_test, y_pred, target_names=all_target_names, labels=[0,1,2,3,4,5], output_dict=True)
        test_classification_reports.append(report)

    mean_val_accuracy = np.mean(val_accuracies)
    mean_test_accuracy = np.mean(test_accuracies)
    print(f"\nAverage Validation Accuracy: {mean_val_accuracy:.4f}")
    print(f"Average Test Accuracy: {mean_test_accuracy:.4f}")

    # In Classification Report
    report_dict = metrics.classification_report(
        all_y_true, all_y_pred,
        target_names=all_target_names,
        labels=[0, 1, 2, 3, 4, 5],
        output_dict=True
    )

    overall_f1 = report_dict["weighted avg"]["f1-score"]
    overall_support = report_dict["weighted avg"]["support"]

    print("\nFinal Overall Classification Report:")
    print(metrics.classification_report(
        all_y_true, all_y_pred,
        target_names=all_target_names,
        labels=[0, 1, 2, 3, 4, 5]
    ))

    print(f"Weighted F1-score: {overall_f1:.4f} | Total Samples: {int(overall_support)}")

    # Save model
    model_filename = f'{model_name.lower()}-aqi-classifier.pkl'
    pickle.dump(best_model, open(model_filename, 'wb'))
    print(f"Best model for {model_name} saved as '{model_filename}'")


▶ Training Model: RandomForest

Processing Fold 1/3
Train Accuracy: 1.0000 | Validation Accuracy: 0.9945 | Test Accuracy: 0.9939

Processing Fold 2/3


/home/thu/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/thu/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/thu/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/thu/miniconda3

Train Accuracy: 1.0000 | Validation Accuracy: 0.9939 | Test Accuracy: 1.0000

Processing Fold 3/3


/home/thu/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/thu/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/thu/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/thu/miniconda3

Train Accuracy: 1.0000 | Validation Accuracy: 1.0000 | Test Accuracy: 1.0000

Average Validation Accuracy: 0.9961
Average Test Accuracy: 0.9980

Final Overall Classification Report:
                         precision    recall  f1-score   support

                   Good       1.00      1.00      1.00       196
               Moderate       1.00      1.00      1.00       380
Unhealthy for Sensitive       0.99      1.00      1.00       161
              Unhealthy       1.00      1.00      1.00       207
         Very Unhealthy       1.00      1.00      1.00         4
              Hazardous       0.00      0.00      0.00         1

               accuracy                           1.00       949
              macro avg       0.83      0.83      0.83       949
           weighted avg       1.00      1.00      1.00       949

Weighted F1-score: 0.9974 | Total Samples: 949


/home/thu/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/thu/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/thu/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/thu/miniconda3

Best model for RandomForest saved as 'randomforest-aqi-classifier.pkl'

▶ Training Model: XGBoost

Processing Fold 1/3


XGBoostError: [20:38:35] /workspace/src/common/common.cu:16: /workspace/src/common/cuda_dr_utils.cc: 24: cudaErrorNoDevice: no CUDA-capable device is detected
Stack trace:
  [bt] (0) /home/thu/miniconda3/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x2a6acc) [0x1554dcea6acc]
  [bt] (1) /home/thu/miniconda3/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0xa59cee) [0x1554dd659cee]
  [bt] (2) /home/thu/miniconda3/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x382c8b) [0x1554dcf82c8b]
  [bt] (3) /home/thu/miniconda3/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x383cfc) [0x1554dcf83cfc]
  [bt] (4) /lib/x86_64-linux-gnu/libc.so.6(+0x99ee8) [0x15555527cee8]
  [bt] (5) /home/thu/miniconda3/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x382bb8) [0x1554dcf82bb8]
  [bt] (6) /home/thu/miniconda3/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0xa5b0c5) [0x1554dd65b0c5]
  [bt] (7) /home/thu/miniconda3/lib/python3.12/site-packages/xgboost